<a href="https://colab.research.google.com/github/uditgaur3/llumo_uat/blob/main/RAGA_UAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00


In [13]:

import json
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load logs
with open("logs.json", "r") as file:
    logs_data = json.load(file)

raga_results = []

# Parse and score
for block in logs_data:
    for item in block["items"]:
        item_id = item["id"]
        system_context = ""
        user_query = ""
        assistant_response = ""

        for i in item["input"]:
            if i["role"] == "system":
                system_context = i.get("context", "")
            elif i["role"] == "user":
                user_query = i.get("context", "")

        for o in item.get("expectedOutput", []):
            if o["role"] == "assistant":
                assistant_response = o.get("content", "")

        try:
            # Cosine similarity: Faithfulness = context vs answer
            faith = util.cos_sim(
                model.encode(system_context),
                model.encode(assistant_response)
            ).item()

            # Cosine similarity: Relevancy = query vs answer
            relev = util.cos_sim(
                model.encode(user_query),
                model.encode(assistant_response)
            ).item()

            raga_results.append({
                "id": item_id,
                "faithfulness": round(faith, 4),
                "answer_relevancy": round(relev, 4)
            })

        except Exception as e:
            print(f"Error for item {item_id}: {e}")
            continue

# Save
output_path = "raga_output.json"
with open(output_path, "w") as f:
    json.dump(raga_results, f, indent=2)

from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>